# Continual Anomaly Detection Experiments

---

### Datasets

We have 2 datasets, MVTec-AD for benchmarking the traditional continual learning methods under the usual framework, where each category/object is its own task, and MTD, which contains one object type, Magnetic Tile Defects.

The magnetic tiles have 5 different kinds of defects, and we will use this dataset to test out how benchmark methods perform in a continous data drift scenario. We will run several experiments on the MTD dataset, where each experiment will have _T_ tasks. The tasks will be progressive disruptions to the image at increasing intensity levels. 

For example, let's say we have an experiment where we see how increasing color jittering (adjusting brightness, saturationm and contrast) over time can be detected accurately, and color jitter can be on an intensity scale of [0, 1]. For task 1, we might jitter the color of images only at intensities within the windowed interval [0, 0.1]. For task 2, we might increase this window to [0, 0.2], and so on. We will be experimenting with different window sizes and whether they overlap or not, and we will save the dataset from our final reported results.

### Experiments

Under a general Continual Learning framework, we assume that a model will experience a series of tasks, where tasks are learned/adapted to one at a time and then tested after learning each task on all previous tasks. 

We will run an experiment to benchmark methods under the current general framework, which uses MVTec-AD and assumes that each category/object is its own task. This is the simplest experiment and is generally used in current literature, in some form or another. This task should be the simplest, because each category has a completely different pixel distribution space. 

For the different disruption types on MTD, we are running an experiment for each type of image distortion:
- __Color Jitter__, which simulates lighting/color changes
- __Blur__, which applies Gaussian Blur and simulates sensor wear or sensor out of focus
- __Geometric__, which applies a mixture of rotation, translation, and shear to simulate product movement or deformation

Again, the idea for these next experiments is that each task will have a similar or overlapping image pixel space, and we can test how well each method learns tasks in a continuous drift manner.

In [ ]:
from Methods.IUF.iuf import IUF_Model
from Methods.DNE.dne import DNE_Model
from utils.train import train_model
from utils.eval import eval_model
import torch

# Hyperparameters for training/testing
torch.manual_seed(42)
# Whether to do training, with which models, and on which datasets
TRAIN = False
EVAL = True
models = {
    "DNE":True,
    "IUF":True,
    "UCAD":False
}
datasets = {
    "MVTEC":True,
    "MTD":True
}
NUM_EPOCHS = 25
BATCH_SIZE = 24
LEARNING_RATE = 0.00025

data_aug = {
    "color": [
        [0.20, 0.26],
        [0.26, 0.32],
        [0.32, 0.38],
        [0.38, 0.44],
        [0.44, 0.50],
        [0.50, 0.56],
        [0.56, 0.62],
        [0.62, 0.68],
        [0.68, 0.74],
        [0.74, 0.80]
    ],
    "blur": [
        [1, 0.5],
        [3, 1],
        [5, 1.5],
        [7, 2],
        [9, 2.5],
        [11, 3],
        [13, 3.5],
        [15, 4],
        [17, 4.5],
        [19, 5],
    ],
    "geometric": [
        [4, 2, 0.02, 4],
        [8, 4, 0.04, 8],
        [12, 6, 0.06, 12],
        [16, 8, 0.08, 16],
        [20, 10, 0.10, 20],
        [24, 12, 0.12, 24],
        [28, 14, 0.14, 28],
        [32, 16, 0.16, 32],
        [36, 18, 0.18, 36],
        [40, 20, 0.2, 40]
    ]
}
# Running Training Experiments
if TRAIN:
    for model in models.keys():
        if models[model]:
            if datasets['MVTEC']:
                train_model(model_type=model,
                            dataset='MVTEC',
                            num_epochs=NUM_EPOCHS,
                            batch_size=BATCH_SIZE,
                            criterion=torch.nn.CrossEntropyLoss(),
                            learning_rate=LEARNING_RATE
                            )
            elif datasets['MTD']:
                for distortion in data_aug.keys():
                    train_model(model_type=model,
                                dataset='MTD',
                                num_epochs=NUM_EPOCHS,
                                batch_size=BATCH_SIZE,
                                criterion=torch.nn.CrossEntropyLoss(),
                                learning_rate=LEARNING_RATE,
                                tasks=data_aug[distortion],
                                data_aug=distortion
                                )

# Running Evaluation Experiments
if EVAL:
    for model in models.keys():
        if models[model]:
            task_data = eval_model(model_type=model,
                                   batch_size=BATCH_SIZE,
                                   data_aug=data_aug
                                   )

In [1]:
from Methods.IUF.iuf import IUF_Model
from utils.train import train_model
import datasets
from utils.eval import eval_model
import torch
from torch.utils.data import DataLoader

iuf = IUF_Model()
test_dataset = datasets.mvtec(train=False, task='bottle', unsupervised=False)
test_dataloader = DataLoader(test_dataset, batch_size=24, shuffle=True, collate_fn=datasets.collate)
out = iuf.eval_one_epoch(test_dataloader)

Using cuda device


In [8]:
for item in out[0]:
    print(item.mean())

tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0067)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0070)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0070)
tensor(-0.0067)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0070)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0067)
tensor(-0.0070)
tensor(-0.0069)
tensor(-0.0069)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0067)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0070)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0067)
tensor(-0.0068)
tensor(-0.0068)
tensor(-0.0069)
tensor(-0.0068)
tensor(-0.0067)
tensor(-0.0067)
tensor(-0.0069)
tensor(-0.0068)
tensor(-

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for exp in ['supervised', 'unsupervised']:
    dataset='MVTEC'
    # Put together accuracy list for each experiment for plotting
    acc_list = []

    # For MVTEC
    if dataset == "MVTEC":
        for task in task_acc[dataset][exp].keys():
            acc_list.append(task_acc[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                 name=f"{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_acc[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_acc[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))


fig.update_layout(title="DNE Accuracy on all Previous Tasks, MVTEC",
                  xaxis_title="Task Number",
                  yaxis_title="Accuracy on all Testing sets of previous tasks")
fig.show()

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for dataset in ["MVTEC", "MTD"]:
    exp='supervised'
    # Put together accuracy list for each experiment for plotting
    acc_list = []
    
    # For MVTEC
    if dataset == "MVTEC":
        for task in task_acc[dataset][exp].keys():
            acc_list.append(task_acc[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines', 
                                 name=f"MVTEC-{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_acc[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_acc[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))
                
        
fig.update_layout(title="DNE Accuracy on all Previous Tasks, Supervised Training",
                  xaxis_title="Task Number",
                  yaxis_title="Accuracy on all Testing sets of previous tasks")
fig.show()

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for dataset in ["MVTEC", "MTD"]:
    exp='unsupervised'
    # Put together accuracy list for each experiment for plotting
    acc_list = []

    # For MVTEC
    if dataset == "MVTEC":
        for task in task_acc[dataset][exp].keys():
            acc_list.append(task_acc[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                 name=f"MVTEC-{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_acc[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_acc[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))


fig.update_layout(title="DNE Accuracy on all Previous Tasks, Unupervised Training",
                  xaxis_title="Task Number",
                  yaxis_title="Accuracy on all Testing sets of previous tasks")
fig.show()

In [ ]:
import numpy as np

task_spec = task_data.copy()
for dataset in task_data.keys():
    for exp in task_data[dataset].keys():
        for task in task_data[dataset][exp].keys():
            (preds, labels) = task_data[dataset][exp][task]
            tn = 0
            fp = 0
            for i in range(len(preds)):
                # 1 is anomaly
                if labels[i]==1:
                    if preds[i]==1: # tn
                        tn += 1
                    else:
                        fp += 1
                        
            specificity = tn / (tn + fp)
            task_spec[dataset][exp][task] = specificity
task_spec

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for exp in ['supervised', 'unsupervised']:
    dataset='MVTEC'
    # Put together accuracy list for each experiment for plotting
    acc_list = []

    # For MVTEC
    if dataset == "MVTEC":
        for task in task_spec[dataset][exp].keys():
            acc_list.append(task_spec[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                 name=f"{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_spec[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_spec[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))


fig.update_layout(title="DNE Sensitivity on all Previous Tasks, MVTEC",
                  xaxis_title="Task Number",
                  yaxis_title="Sensitivity on all Testing sets of previous tasks")
fig.show()

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for dataset in ["MVTEC", "MTD"]:
    exp='supervised'
    # Put together accuracy list for each experiment for plotting
    acc_list = []

    # For MVTEC
    if dataset == "MVTEC":
        for task in task_spec[dataset][exp].keys():
            acc_list.append(task_spec[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                 name=f"MVTEC-{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_spec[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_spec[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))


fig.update_layout(title="DNE Sensitivity on all Previous Tasks, Supervised Training",
                  xaxis_title="Task Number",
                  yaxis_title="Sensitivity on all Testing sets of previous tasks")
fig.show()

In [ ]:
# Plot figure comparing experiments
import plotly.graph_objects as go

fig = go.Figure()
for dataset in ["MVTEC", "MTD"]:
    exp='unsupervised'
    # Put together accuracy list for each experiment for plotting
    acc_list = []

    # For MVTEC
    if dataset == "MVTEC":
        for task in task_spec[dataset][exp].keys():
            acc_list.append(task_spec[dataset][exp][task])
        fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                 name=f"MVTEC-{exp}"))
    # For MTD
    elif dataset == "MTD":
        # Iterate through each data aug type
        for aug in ['color', 'blur', 'geometric']:
            acc_list = []
            for task in task_spec[dataset][exp].keys():
                if task.startswith(aug):
                    acc_list.append(task_spec[dataset][exp][task])
            fig.add_trace(go.Scatter(y=acc_list, mode='lines',
                                     name=f"MTD-{exp}-{aug}"))


fig.update_layout(title="DNE Sensitivity on all Previous Tasks, Unupervised Training",
                  xaxis_title="Task Number",
                  yaxis_title="Sensitivity on all Testing sets of previous tasks")
fig.show()

In [ ]:
np.count_nonzero((preds+labels)==2)